# Chapter 3 - Exercise 6
#### Author: *John Benedick Estrada*
---
**Exercise:** Dijkstra’s algorithm solves the “single source shortest path” problem, but to compute the characteristic path length of a graph, we actually want to solve the “all pairs shortest path” problem.

Of course, one option is to run Dijkstra’s algorithm n times, once for each starting node. And for some applications, that’s probably good enough. But there are are more efficient alternatives.

Find an algorithm for the all-pairs shortest path problem and implement it. See [http://thinkcomplex.com/short.
](https://en.wikipedia.org/wiki/Shortest_path_problem#All-pairs_shortest_paths).

Compare the run time of your implementation with running Dijkstra’s algorithm n times. Which algorithm is better in theory? Which is better in practice? Which one does NetworkX use?

In [1]:
import networkx as nx
from math import inf
from collections import deque
from typing import Hashable, Iterator

##### Function for generating ring lattices
Source: https://github.com/AllenDowney/ThinkComplexity2/blob/master/notebooks/chap03.ipynb

In [2]:
# `adjacent_edges` in Cell 3 of `chap03.ipynb`
# Type hints were added by me.
def adjacent_edges(nodes: Iterator, halfk: int):
    n = len(nodes)
    for i, u in enumerate(nodes):
        for j in range(i+1, i+halfk+1):
            v = nodes[j % n]
            yield u, v


# `make_ring_lattice` in Cell 5 of `chap03.ipynb`
# Type hints were added by me.
def make_ring_lattice(n: int, k: int):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from(adjacent_edges(nodes, k//2))
    return G


# `shortest_path_dijkstra` in Cell 44 of `chap03.ipynb`
# Type hints were added by me.
def shortest_path_dijkstra(G: nx.Graph, source: Hashable):
    dist = {source: 0}
    queue = deque([source])
    while queue:
        node = queue.popleft()
        new_dist = dist[node] + 1

        neighbors = set(G[node]).difference(dist)
        for n in neighbors:
            dist[n] = new_dist
        
        queue.extend(neighbors)
    return dist

##### Dijkstra's shortest path algorithm for all pairs

In [3]:
def dijkstra_spa_all_pairs(G: nx.Graph):
    dist = dict((node, inf) for node in G)

    for source in G.nodes():
        dist[source] = shortest_path_dijkstra(G, source)
    return dist

##### Floyd-Warshall algorithm

The pseudo-codes used as basis for my implementation are found [here](https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm) and [here](https://www.geeksforgeeks.org/floyd-warshall-algorithm-dp-16/).

In [4]:
def floyd_warshall(G: nx.Graph):
    nodes = G.nodes
    dist = dict((n, {}) for n in nodes)
    # Populate `dist` with known lengths between adjacent nodes.
    for n in nodes:
        neighbors = G[n]
        for m in nodes:
            if m == n:
                dist[n][m] = 0
            elif m in neighbors:
                dist[n][m] = 1  # We treat all edge "weights" in unweighted graphs as 1.
            else:
                dist[n][m] = inf

    for k in nodes:
        for i in nodes:
            for j in nodes:
                dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])

    return dist

##### Test the Floyd-Warshall implementation

In [5]:
n = 30
for k in range(2, n+2, 2):
    G = make_ring_lattice(n, k)
    assert dijkstra_spa_all_pairs(G) == floyd_warshall(G), \
        f"Test failed at k={k} for n={n}"

print("Success!")
del G

Success!


##### Benchmark the two all-pair shortest path algorithms

In [6]:
G = make_ring_lattice(10, 5)

print("Modified Dijkstra's SPA:  ", end="")
%timeit dijkstra_spa_all_pairs(G)

Modified Dijkstra's SPA:  211 µs ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
G = make_ring_lattice(10, 5)

print("Floyd-Warshall algorithm: ", end="")
%timeit floyd_warshall(G)

Floyd-Warshall algorithm: 475 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
